In [37]:
# read_csv from all files in folder Zero2Heros/data/all year/
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


# read csv file
#germ_data = pd.read_csv('/Users/anatolijkekarev/Desktop/jupyter_env/Zeros2Heros/data/all year/gen_DE_B01.csv', sep=',')

In [40]:
# list all the files the folder /Users/anatolijkekarev/Desktop/jupyter_env/Zeros2Heros/data/all year
path = '/Users/anatolijkekarev/Desktop/jupyter_env/Zeros2Heros/data/some year/'
files = os.listdir(path)
files

['gen_DE_B19.csv', 'gen_SE_B12.csv', 'load_UK.csv']

In [50]:
limbo = pd.DataFrame()

def transform(x):
    # Your transformation logic here
    return pd.to_datetime(x)

def clean_data(name):
    # drop columns
    germ_data = pd.read_csv(f'/Users/anatolijkekarev/Desktop/jupyter_env/Zeros2Heros/data/all year/{name}', sep=',')
    print(f'applying the clean_data function to the file: {name}')
    if name.startswith('load'):
        #rename the column 'Load' to 'quantity'
        germ_data.rename(columns={'Load':'quantity'}, inplace=True)

    germ_data = germ_data[['quantity','StartTime']]
    germ_data = germ_data.drop(0)
    # Apply the split operation to the 'StartTime' column
    germ_data['StartTime'] = germ_data['StartTime'].apply(lambda s: s.split('+')[0])
    # Apply the transformation to the 'StartTime' column
    germ_data['StartTime'] = germ_data['StartTime'].apply(transform)
    germ_data.index = germ_data['StartTime']
    germ_data.drop('StartTime',axis=1,inplace=True)
    germ_data['quantity'] = germ_data_sel['quantity'].astype(int)
    germ_data = germ_data.resample('1H').sum()
    new_name = name.replace('.csv', '').replace('load', 'loa')
    limbo[new_name] = germ_data['quantity']
    print('data appended to limbo')


In [51]:
for file in files:
    clean_data(name = file)

applying the clean_data function to the file: gen_DE_B19.csv
data appended to limbo
applying the clean_data function to the file: gen_SE_B12.csv
data appended to limbo
applying the clean_data function to the file: load_UK.csv
data appended to limbo


In [53]:
limbo.isna().sum()

gen_DE_B19       0
gen_SE_B12       2
loa_UK        4009
dtype: int64

In [54]:
limbo.loa_UK.max()

10090.0

In [16]:
germ_data = germ_data[['quantity','StartTime']]

# Apply the split operation to the 'StartTime' column
germ_data['StartTime'] = germ_data['StartTime'].apply(lambda s: s.split('+')[0])

# Apply the split operation to the 'EndTime' column
germ_data['EndTime'] = germ_data['EndTime'].apply(lambda s: s.split('+')[0])

# Define your transformation function
def transform(x):
    # Your transformation logic here
    return pd.to_datetime(x)
# Apply the transformation to the 'StartTime' column
germ_data['StartTime'] = germ_data['StartTime'].apply(transform)
germ_data['EndTime'] = germ_data['EndTime'].apply(transform)
germ_data.drop(0,inplace=True)

germ_data.info()

SyntaxError: invalid syntax (3876698159.py, line 17)

In [59]:
germ_data = pd.read_csv('/Users/anatolijkekarev/Desktop/jupyter_env/Zeros2Heros/data/all year/load_UK.csv', sep=',')
germ_data = germ_data.drop(0)

germ_data.rename(columns={'Load':'quantity'}, inplace=True)
germ_data = germ_data[['quantity','StartTime']]
germ_data_sel['quantity'] = germ_data_sel['quantity'].astype(int)
germ_data['StartTime'] = germ_data['StartTime'].apply(lambda s: s.split('+')[0])

germ_data['StartTime'] = germ_data['StartTime'].apply(transform)
germ_data.resample('1H', on = 'StartTime').sum()


,quantity
StartTime,
2022-01-01 00:00:00,1244
2022-01-01 01:00:00,1131
2022-01-01 02:00:00,1091
2022-01-01 03:00:00,969
2022-01-01 04:00:00,896
...,...
2022-07-17 19:00:00,786
2022-07-17 20:00:00,759
2022-07-17 21:00:00,1469


In [58]:
germ_data.isna().sum()

quantity     0
StartTime    0
dtype: int64

In [61]:
germ_data.iloc[-1,:]

quantity                     603
StartTime    2022-07-17 23:00:00
Name: 9339, dtype: object

In [29]:
# i would like you to help me with data summation 
# i want to sum the data for each hour 
# that requires grouping by hour and summing the quantity column  for each hour
germ_data germ_data['EndTime'][35038].minute

30